Script to quantify error induced by using TMY weather with actual year load in a variety of locations with a variety of utility rate structures

Requires:
nrel-pysam
requests
numpy
pandas

In [40]:
import numpy as np
import pandas as pd
import json
import os

import PySAM.Battery as battery_model
import PySAM.Pvsamv1 as pv_model
import PySAM.Utilityrate5 as utility_rate
import PySAM.Cashloan as cashloan
import PySAM.ResourceTools
import PySAM.UtilityRateTools

In [ ]:
def get_pysam_json(json_file_path):
    """
    Open a PySAM JSON file and return as a dictionary
    """
    with open(json_file_path) as f:
        dic = json.load(f)
    return dic


def get_load_profile(load_path, desired_timestep_minutes):
    """
    Get data out of a CSV file
    Original ResStock data is 15 minute kWh data - need hourly for TMY comparison and to convert to kW
    """
    df = pd.read_csv(load_path)
    timeseries = df["out.electricity.total.energy_consumption"].values
    
    if desired_timestep_minutes < 15:
        raise ValueError("get_load_profile is not set up for timesteps less than 15 minutes.")
    elif desired_timestep_minutes == 15:
        return timeseries * 4 # Convert from kWh to kW
    elif desired_timestep_minutes % 15 != 0: 
        raise ValueError("get_load_profile is not set up for that aren't evenly divisiable by 15. Pick 15, 30, or 60.")
    else:
        averaged_timesteps = []
        steps_per_step = desired_timestep_minutes / 15
        steps_per_hr = 60 / desired_timestep_minutes
        step = 0
        avg_kwhs = 0
        for kwh in timeseries:
             avg_kwhs += kwh
             step += 1
             if step == steps_per_step:
                 averaged_timesteps.append(avg_kwhs * steps_per_hr)
                 step = 0
                 avg_kwhs = 0
        return averaged_timesteps

In [43]:
def run_location(weather_file, rate_path, load_path):

    # Utility rate data is contained here since the battery needs it for dispatch calculations
    rate_setup = get_pysam_json(rate_path)
    wf_timestep = 60
    load_profile = get_load_profile(load_path, wf_timestep)

    pv = pv_model.default("PVBatteryResidential") # This runs both PV and battery
    ur = utility_rate.from_existing(pv, "PVBatteryResidential")
    cl = cashloan.from_existing(ur, "PVBatteryResidential")

    for k, v in rate_setup.items():
        try:
            pv.value(k, v)
        except AttributeError:
            if "batt_adjust" in k:
                pass
            else:
                print("Failed to assign PV key " + str(k))

    pv.value("solar_resource_file", str(weather_file))
    pv.value("batt_dispatch_choice", 4) # Retail rates dispatch
    pv.value("load", load_profile)

    pv.execute()
    ur.execute()
    cl.execute()

    output_data = {}
    output_data.update(pv.Outputs.export())
    output_data.update(ur.Outputs.export())
    output_data.update(cl.Outputs.export())
    return output_data


To generate rate data:

- Open the SAM file in sam files
- Download the rate data on the utility rates page
- Make adjustments as needed (e.g. "net billing" for California rates)
- Use shift-F5 to export the code and choose "PySAM JSON"
- Save the untitled_pvsamv1.json file to the rate_data folder and rename it to something more descriptive

In [53]:
file_dir = os.path.abspath('')

rate_path = file_dir + "/rate_data/" + "jefferson_county_co_tou_pvsamv1.json"

weather_path = file_dir + "/weather_data/" + "39.81_-105.14_nsrdb-GOES-tmy-v4-0-0_60_tmy.csv"

load_path = file_dir + "/load_data/" + "load-data-total-10388-0.csv"

tmy_outputs = run_location(weather_path, rate_path, load_path)


weather_path = file_dir + "/weather_data/" + "39.81_-105.14_nsrdb-GOES-aggregated-v4-0-0_60_2018.csv"

amy_outputs = run_location(weather_path, rate_path, load_path)

In [54]:
print(tmy_outputs["utility_bill_w_sys"][1])

print(amy_outputs["utility_bill_w_sys"][1])


-1188.7831844304023
-1228.5869349588622


In [51]:
load_profile = get_load_profile(load_path, 15)

print(sum(load_profile[0:8760 * 4]) / 4)

4106.631250000069
